# Generalized Assignment Problem
Un conjunto de $n$ trabajos deben realizarse por $m$ máquinas, que son distintas entre si.

Si realizo el trabajo $i$ en la maquina $j$, este va a requerir un tiempo $d_{ij}$ (``duracion[i,j]``) para realizarse, y obtendrá un beneficio $b_{ij}$ (``beneficio[i,j]``) por realizarse, para todo $i=1\ldots n, j=1\ldots m$ 

Cada máquina tiene una cantidad fija de tiempo disponible para realizar trabajos $c_j$ (``disponible[j]``) para todo $j=1\ldots m$.

El objetivo del problema es decidir que trabajos realizar, y en que máquina realizarlos, de forma de maxímizar el beneficio obtenido sin violar la duración máxima de cada máquina.

**Veamos primero los datos:**

In [ ]:
#Numero de trabajos
n = 50
#Número de maquinas
m = 5

In [ ]:
# Duracion de cada trabajo en cada maquina
duracion = rand(n,m)*9+1
# beneficio por procesar cada trabajo en cada maquina
beneficio = randexp(n,m)
# Tiempo disponible de cada maquina
disponible = rand(m) * n/m * 10 * 0.5

El modelo más clásico es formular usando una variable binaria $z_{ij}$ si el trabajo $i$ se asigna a la máquina $j$. De esta forma el problema puede ser modelado como:
$$
\begin{align}
\max \sum_{i=1}^n \sum_{j=1}^m b_{ij} z_{ij}&\\
\sum_{j=1}^m z_{ij} \leq 1&\quad i=1\ldots n\\
\sum_{i=1}^n d_{ij} z_{ij} \leq c_j&\quad j=1\ldots m\\
z_{ij} \in\{0,1\} &\quad i=1\ldots n, j=1\ldots m\\
\end{align}
$$
donde el objetivo es maximizar el beneficio, la primera restricción asegura que cada trabajo se realiza a lo mas una vez, y la segunda restricción asegura que la duración total de las tareas asignadas a cada máquina no excede su capacidad.

El problema puede ser formulado en JuMP de la siguiente forma:

In [ ]:
using JuMP
using Clp 
using Cbc

In [ ]:
model = Model(solver=CbcSolver())
@variable(model, z[1:n,1:m] >= 0, Bin)
for i=1:n
    @constraint(model, sum(z[i,j] for j=1:m) <= 1) 
end
for j=1:m
    @constraint(model, sum(duracion[i,j]*z[i,j] for i=1:n) <= disponible[j]) 
end
@objective(model, Max, sum(z[i,j]*beneficio[i,j] for i=1:n for j=1:m));

In [ ]:
solve(model)

In [ ]:
epsilon = 1e-6
sol=getvalue(z)
for i =1:n
    for j=1:m
        if sol[i,j] > epsilon
            println("Trabajo ",i," asignado a maquina ",j)
        end
    end
end 
println("Objetivo es: ", getobjectivevalue(model))

# Formulación Alternativa
Otra forma de resolver este problema es suponiendo que tenemos un conjunto $\mathcal{C}_m$ con todas las posibles combinaciones de trabajos que tienen una duración total válida para la máquina $m$. En ese caso, el problema _maestro_ se podría escribir como:
$$
\begin{align}
\max \sum_{j=1}^m \sum_{k\in\mathcal{C}_j} p_{k} x_{kj}&\\
\sum_{j=1}^m \sum_{k\in\mathcal{C}_j}a_{ki} x_{kj} \leq 1&\quad i=1\ldots n\\
\sum_{k\in\mathcal{C}_j}  x_{kj} = 1&\quad j=1\ldots m\\
x_{kj} \in\{0,1\} &\quad k\in\mathcal{C}_j, j=1\ldots m\\
\end{align}
$$
donde $p_{kj}$ es el beneficio total de asignar la combinación $k$ en la maquina $j$, y 
$$
a_{ki} = \begin{cases} 1 & \text{ si $i$ está incluido en la combinación $k$} \\ 0 & \text {si no} \end{cases}
$$
El modelo maximiza el beneficio total asignado, con las restricciones de que cada trabajo $i$ queda asignado a una máquina, y que cada máquina $j$ tiene exactamente una combinación asignada.

Como obviamente el número de variables es muy grande, podemos usar generación es columnas. Es decir, partimos con un número limitado de variables (por ejemplo, una sola combinación con $a_{1i}=0\ \forall i,j)$, y las vamos agregando en la medida que las necesitemos. Para determinar si es necesario agregar una nueva variable, obtenemos los duales $\pi_i$ y $\mu_j$ del primer y segundo grupo de restricciones respectivamente. De esta forma, podemos encontrar la combinación de mayor costo reducido resolviendo un problema *Pricing* para cada máquina $j=1\ldots m$, resolviendo el problema:
$$
\begin{align}
\max \sum_{i=1}^n (b_{ij} - \pi_i) y_{i}&\\
\sum_{i=1}^n d_{ij} y_{i} \leq c_j\\
y_{i} \in\{0,1\} &\quad i=1\ldots n\
\end{align}
$$
y si el valor de objetivo de este problema es mayor a $\mu_j$, se agrega esa variable como una combinación posible para la maquina $j$.

## Parte 1 : Master
Formule el problema _master_, a partir de un conjunto de combinaciones dado. Resuelva el problema y recupere los duales de cada restricción. Recuerde que debe usar `ClpSolver()` para poder recuperar los duales del problema.

In [ ]:
combinaciones=[]
push!(combinaciones,zeros(n))

## Parte 2: Pricing
Formule el problema _pricing_ usando los duales obtenido en el problema _master_.

## Parte 3 : Generación de Columnas
Incorpore el problema _master_ y _pricing_ en un loop, de forma de generar todas las columnas que sean necesarias para llegar al óptimo del problema.

### Nota teórica:
Si todo es correcto, debería obtener el mismo valor objetivo que en la formulación inicial. Esto es porque el problema _master_ tiene vértices enteros, por lo que aun si estamos resolviendo una relajación lineal, la solución es entera!

## Parte 4 ¿Es mejor Generación de Columnas o la formulación inicial?
Verifique remplazando los números iniciales de trabajos $n$ y de máquinas $m$, por otros mas grandes para ver si generación de columnas es igual, peor o similar a la formulación inicial. 
Responda sus hallazgos en la siguiente ventana.